# Lab 4: Collaborative Filtering Recommender Systems

This lab implements the collaborative filtering learning algorithm and applies it to a dataset of movie ratings. 

The goal of a collaborative filtering recommender system is to generate two vectors: 
* For each user, a 'parameter vector' that embodies the movie tastes of a user.
* For each movie, a feature vector of the same size which embodies some description of the movie.
* The dot product of the two vectors plus the bias term should produce an estimate of the rating the user might give to that movie.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from lib.recsys_utils import *

## Load Data

*The data set is derived from the [MovieLens "ml-latest-small"](https://grouplens.org/datasets/movielens/latest/) dataset.   
[F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>]*

We use a subset of the original dataset.
* The original dataset has 9000 movies rated by 600 users.
* The dataset has been reduced in size to focus on movies from the years since 2000.
* The reduced dataset has $n_u = 443$ users, and $n_m= 4778$ movies.

Aspects of the dataset:
* Existing ratings are provided in matrix Y. Ratings range from 0.5 to 5 inclusive in 0.5 steps. 0 if the movie has not been rated.
* R matrix indicates whether or not a user has rated a movie. 1 for rated. Movies are in rows, users in columns.
* Each user has a parameter vector (W)and bias (b).
* Each movie has a feature vector (X). These vectors are simultaneously learned by using the existing user/movie ratings as training data. Once the feature vectors and parameters are learned, they can be used to predict how a user might rate an unrated movie.

In [2]:
#Load data
X, W, b, num_movies, num_features, num_users = load_precalc_params_small()
Y, R = load_ratings_small()

print("Y", Y.shape, "R", R.shape)
print("X", X.shape)
print("W", W.shape)
print("b", b.shape)
print("num_features", num_features)
print("num_movies",   num_movies)
print("num_users",    num_users)

Y (4778, 443) R (4778, 443)
X (4778, 10)
W (443, 10)
b (1, 443)
num_features 10
num_movies 4778
num_users 443


In [3]:
#  From the matrix, we can compute statistics like average rating.
tsmean =  np.mean(Y[0, R[0, :].astype(bool)])
print(f"Average rating for movie 1 : {tsmean:0.3f} / 5" )

Average rating for movie 1 : 3.400 / 5


## Implement loss function for collaborative filtering

This section implements the function `cofiCostFunc` that computes the collaborative filtering objective function. After implementing the objective function, we use a TensorFlow custom training loop to learn the parameters for collaborative filtering.

### Without vectorization or regularization

In [35]:
# GRADED FUNCTION: cofi_cost_func
# UNQ_C1

def cofi_cost_func_not_regularized(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
      
    Returns:
      J (float) : Cost
    """
    nm, nu = Y.shape # Y.shape: movie_count, user_count
    J = 0
    ### START CODE HERE ###  
    
    # Summation round 1 (over each user)
    for i in range(nu):
        w = W[i, :]
        b_i = b[0, i]

        # Summation round 2 (over each movie)
        for j in range(nm):
            x = X[j, :] 
            y = Y[j, i] # labelled rating
            r = R[j, i] # used to turn on/off the cost because 0 if this movie not rated by this user

            # Make prediction of review for this user/movie combination.
            y_hat = np.dot(w, x) + b_i       

            # Calculate loss as the squared difference between the prediction (y_hat) to the actual review (y)
            squared_loss = (r * (y_hat - y))**2

            # Add the loss for this user/movie combination to the overall loss
            J += squared_loss

    # Why are we dividing by 2 instead of dividing by 2*count?
    J /= 2
    ### END CODE HERE ### 

    return J

In [43]:
# TEST THE CUSTOM NON-REGULARIZED FUNCTION

# Reduce the data set size so that this runs faster
num_users_r = 4
num_movies_r = 5 
num_features_r = 3

X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r,  :num_features_r]
b_r = b[0, :num_users_r].reshape(1,-1)
Y_r = Y[:num_movies_r, :num_users_r]
R_r = R[:num_movies_r, :num_users_r]

# Evaluate cost function
J = cofi_cost_func_not_regularized(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost (without regularization): {J:0.2f}")

# Expected Cost when lambda = 0 (no regularization): 13.67

Cost (without regularization): 13.67


### Without vectorization, with regularization

In [44]:
# GRADED FUNCTION: cofi_cost_func
# UNQ_C1

# Regularized Version
def cofi_cost_func(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
      
    Returns:
      J (float) : Cost
    """
    nm, nu = Y.shape # Y.shape: movie_count, user_count
    J = 0
    ### START CODE HERE ###  
    
    # Summation round 1 (over each user)
    for i in range(nu):
        w = W[i, :]
        b_i = b[0, i]

        # Summation round 2 (over each movie)
        for j in range(nm):
            x = X[j, :] 
            y = Y[j, i] # labelled rating
            r = R[j, i] # used to turn on/off the cost because 0 if this movie not rated by this user

            # Make prediction of review for this user/movie combination.
            y_hat = np.dot(w, x) + b_i       

            # Calculate loss as the squared difference between the prediction (y_hat) to the actual review (y)
            squared_loss = (r * (y_hat - y))**2

            # Add the loss for this user/movie combination to the overall loss
            J += squared_loss

    # Why are we dividing by 2 instead of dividing by 2*count?
    J /= 2

    # To regularize the data: 
    #    Square each element of the W array and X array. 
    #    Then sums all the squared elements. 
    # In previous work this was implemented with a loop but we can use numpy to vectorize the implementation.
    J += (lambda_/2) * (np.sum(np.square(W)) + np.sum(np.square(X)))

    ### END CODE HERE ### 

    return J

In [45]:
# TEST THE CUSTOM REGULARIZED FUNCTION

# Reduce the data set size so that this runs faster
num_users_r = 4
num_movies_r = 5 
num_features_r = 3

X_r = X[:num_movies_r, :num_features_r]
W_r = W[:num_users_r,  :num_features_r]
b_r = b[0, :num_users_r].reshape(1,-1)
Y_r = Y[:num_movies_r, :num_users_r]
R_r = R[:num_movies_r, :num_users_r]

# Evaluate cost function WITHOUT REGULARIZATION (lambda = 0)
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost (without regularization): {J:0.2f}")
# Expected Cost without regularization: 13.67


# Evaluate cost function WITH REGULARIZATION (lambda = 1.5) 
J = cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

# Should be higher than 13.67 (cost without regularization) because the regularization prevents over-fitting to the training set.
# Expected Output: 28.09 

Cost (without regularization): 13.67
Cost (with regularization): 28.09


In [46]:
# Public tests
from public_tests import *
test_cofi_cost_func(cofi_cost_func)

All tests passed!


### With vectorization and regularization

This is important because we will call the method repeatedly to optimize the parameters / minimize loss.

In [50]:
def cofi_cost_func_vectorized(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    Args:
      X (ndarray (num_movies,num_features)): matrix of item features
      W (ndarray (num_users,num_features)) : matrix of user parameters
      b (ndarray (1, num_users)            : vector of user parameters
      Y (ndarray (num_movies,num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies,num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float): regularization parameter
    Returns:
      J (float) : Cost
    """
    j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
    return J

In [51]:
# Evaluate cost function with vectorization but without regularization (lambda=0)
J = cofi_cost_func_vectorized(X_r, W_r, b_r, Y_r, R_r, 0);
print(f"Cost (without regularization): {J:0.2f}")
# Expected Output: 13.67

# Evaluate cost function with vectorization and regularization 
J = cofi_cost_func_vectorized(X_r, W_r, b_r, Y_r, R_r, 1.5);
print(f"Cost (with regularization): {J:0.2f}")
# Expected Output: 28.09

Cost (without regularization): 13.67
Cost (with regularization): 28.09


## Load data

In [59]:
movieList, movieList_df = load_Movie_List_pd()

my_ratings = np.zeros(num_movies)          #  Initialize my ratings

# Check the file small_movie_list.csv for id of each movie in our dataset
# For example, Toy Story 3 (2010) has ID 2700, so to rate it "5", you can set
my_ratings[2700] = 5 

#Or suppose you did not enjoy Persuasion (2007), you can set
my_ratings[2609] = 2;

# We have selected a few movies we liked / did not like and the ratings we
# gave are as follows:
my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
my_ratings[246]  = 5   # Shrek (2001)
my_ratings[2716] = 3   # Inception
my_ratings[1150] = 5   # Incredibles, The (2004)
my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)
my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

print('\nNew user ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0 :
        print(f'Rated {my_ratings[i]} for  {movieList_df.loc[i,"title"]}');


New user ratings:

Rated 5.0 for  Shrek (2001)
Rated 5.0 for  Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Rated 2.0 for  Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Rated 5.0 for  Harry Potter and the Chamber of Secrets (2002)
Rated 5.0 for  Pirates of the Caribbean: The Curse of the Black Pearl (2003)
Rated 5.0 for  Lord of the Rings: The Return of the King, The (2003)
Rated 3.0 for  Eternal Sunshine of the Spotless Mind (2004)
Rated 5.0 for  Incredibles, The (2004)
Rated 2.0 for  Persuasion (2007)
Rated 5.0 for  Toy Story 3 (2010)
Rated 3.0 for  Inception (2010)
Rated 1.0 for  Louis Theroux: Law & Disorder (2008)
Rated 1.0 for  Nothing to Declare (Rien à déclarer) (2010)


In [60]:
# Reload ratings
Y, R = load_ratings_small()

# Add new user ratings to Y 
Y = np.c_[my_ratings, Y]

# Add new user indicator matrix to R
R = np.c_[(my_ratings != 0).astype(int), R]

# Normalize the Dataset
Ynorm, Ymean = normalizeRatings(Y, R)

## Train model

We will use tensorflow and the manually coded cost function in the previous section to optimize the paremeters (X, W, b) by mininimizing the costfunction.

In [61]:
#  Useful Values
num_movies, num_users = Y.shape
num_features = 100

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [62]:
iterations = 200
lambda_ = 1
for iter in range(iterations):
    
    # Use Tensorflow's GradientTape to record the steps of calculating the cost function (J).
    # This tape enables differentiation... which we need in the subsequent step to isolate the steps of gradient descent.
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        # When using collaborative filtering, the cost function has three variables (instead of two)
        # ...weights (W) bias (b) and features (X). Collaborative filtering deduces ALL of these.
        cost_value = cofi_cost_func_vectorized(X, W, b, Ynorm, R, lambda_)

    # Use the tape to retrieve the gradients/derivatives (with respect to X W and b)
    # These derivatives will be used in the next line to perform one iteration of gradient descent.
    gradients = tape.gradient( cost_value, [X, W, b] )

    # Run one step of gradient descent; This updates the value of w (reducing the cost, stepping down a valley).
    # Reminder on the process of gradient descent...the following steps happen at each iteration
    #    - Calculate the derivative with respect to w and then update w by learning_rate * derivative
    #    - Calculate the derivative with respect to b and then update b by learning_rate * derivative
    #    - Calculate the derivative with respect to x and then update x by learning_rate * derivative (unique to collaborative filtering)
    optimizer.apply_gradients( zip(gradients, [X, W, b]) )

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")


# Expected output on first run: Training loss at iteration 180: 2902.1
# NB: If you run this cell multiple times, the algorithm will continue to optimize the parameters and lower the loss.

Training loss at iteration 0: 2321191.3
Training loss at iteration 20: 136169.3
Training loss at iteration 40: 51863.7
Training loss at iteration 60: 24599.0
Training loss at iteration 80: 13630.6
Training loss at iteration 100: 8487.7
Training loss at iteration 120: 5807.8
Training loss at iteration 140: 4311.6
Training loss at iteration 160: 3435.3
Training loss at iteration 180: 2902.1


## Predict reviews / Make recommendations

We compute the ratings for all the movies and users and display the movies that are recommended. These are based on the movies and ratings entered as my_ratings[] above.

In [63]:
# Make a prediction using trained weights and biases (trained in previous section)
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

#restore the mean
pm = p + Ymean

my_predictions = pm[:,0]

# sort predictions
ix = tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movieList[j]}')

print('\n\nOriginal vs Predicted ratings:\n')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print(f'Original {my_ratings[i]}, Predicted {my_predictions[i]:0.2f} for {movieList[i]}')

Predicting rating 4.49 for movie My Sassy Girl (Yeopgijeogin geunyeo) (2001)
Predicting rating 4.48 for movie Martin Lawrence Live: Runteldat (2002)
Predicting rating 4.48 for movie Memento (2000)
Predicting rating 4.47 for movie Delirium (2014)
Predicting rating 4.47 for movie Laggies (2014)
Predicting rating 4.47 for movie One I Love, The (2014)
Predicting rating 4.46 for movie Particle Fever (2013)
Predicting rating 4.45 for movie Eichmann (2007)
Predicting rating 4.45 for movie Battle Royale 2: Requiem (Batoru rowaiaru II: Chinkonka) (2003)
Predicting rating 4.45 for movie Into the Abyss (2011)


Original vs Predicted ratings:

Original 5.0, Predicted 4.90 for Shrek (2001)
Original 5.0, Predicted 4.84 for Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Original 2.0, Predicted 2.13 for Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)
Original 5.0, Predicted 4.88 for Harry Potter and the Chamber of Secrets (2002)
Original 5.0, Predic

## Visualization

Above, the predicted ratings for the first few hundred movies lie in a small range. We can augment the above by selecting from those top movies, movies that have high average ratings and movies with more than 20 ratings. This section uses a Pandas data frame which has many handy sorting features.

In [64]:
filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_predictions
movieList_df = movieList_df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)

,pred,mean rating,number of ratings,title
1743,4.030961,4.252336,107,"Departed, The (2006)"
2112,3.985281,4.238255,149,"Dark Knight, The (2008)"
211,4.477798,4.122642,159,Memento (2000)
929,4.887054,4.118919,185,"Lord of the Rings: The Return of the King, The..."
2700,4.796531,4.109091,55,Toy Story 3 (2010)
653,4.357304,4.021277,188,"Lord of the Rings: The Two Towers, The (2002)"
1122,4.004471,4.006494,77,Shaun of the Dead (2004)
1841,3.980649,4.000000,61,Hot Fuzz (2007)
3083,4.084643,3.993421,76,"Dark Knight Rises, The (2012)"
2804,4.434171,3.989362,47,Harry Potter and the Deathly Hallows: Part 1 (...
